In [3]:

import os
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame
from loguru import logger
from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *


import os
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame
from loguru import logger
from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *


import uuid
import traceback
from datetime import datetime
from llm_web_kit.input.datajson import DataJson
from func_timeout import FunctionTimedOut, func_timeout
import os
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame
from loguru import logger
from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *

from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
import re



from urllib.parse import urlparse


import uuid
import traceback
from datetime import datetime
from llm_web_kit.input.datajson import DataJson
from func_timeout import FunctionTimedOut, func_timeout


In [4]:

def transform_row(row, config: dict) -> Row:

    """根据平台配置转换行数据"""
    mappings = config
    from loguru import logger
    try:

         # 获取已使用的字段名
        used_field_names = {
            mappings["track_id"],
            mappings["url"],
            mappings["html"],
            mappings["layout_field"],
            "filename"  # 显式使用的filename字段
        }

        # 获取原始行的所有字段名
        row_field_names = row.__fields__  # 适用于pyspark.sql.Row对象

        # 收集未被使用的字段
        remaining_fields = [f for f in row_field_names if f not in used_field_names]
  
        # 构建meta_info字典
        meta_info = {"filename": getattr(row, "filename", "")}
        for field in remaining_fields:
            meta_info[field] = getattr(row, field, None)

        d = Row(
                track_id=getattr(row, mappings["track_id"],str(uuid.uuid4())),
                url=getattr(row, mappings["url"], ''),
                html=getattr(row, mappings["html"], ''),
                page_layout_type=mappings.get("page_layout_type_map").get(
                    getattr(row, mappings["layout_field"], ''),
                    "article"
                ),
                domain=extract_domain_info(getattr(row, mappings["url"], ''))['domain'],
                dataset_name=mappings["dataset_name"],
                data_source_category=mappings["data_source_category"],
                meta_info=meta_info
            )
        return d 
    except Exception as e:
        # 记录更详细的错误信息
        error_info = {
            "error_data":Row(**d),
            "error_type": type(e).__name__,
            "error_message": str(e),
            "traceback": traceback.format_exc(),
        }
        # s3_doc_writer.write(error_info)
        d['__error'] = error_info
    
        return Row(**d)
   


def extract_data(partition, broadcast_extractor_config_path):
    from llm_web_kit.extractor.extractor_chain import ExtractSimpleFactory
    from loguru import logger

    extractor_chain = ExtractSimpleFactory.create('/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc')
    timeout_seconds = 10
    # 为每个分区创建唯一的错误日志文件
    # partition_id = str(uuid.uuid4())
    # current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    # error_log_path = f"s3://xyz-llm-users/xyz-users/yujia/CC-MAIN-2024-33/output/v002/error_logs/{current_time}_{partition_id}.json"
    # s3_doc_writer = S3DocWriter(path=error_log_path)
    for row in partition:
        try:

            d = row.asDict()
            input_data = DataJson(d)
            data_e: DataJson = func_timeout(timeout_seconds, extractor_chain.extract, args=(input_data,))
            #data_e: DataJson = extractor_chain.extract(input_data)
            
    
            yield Row(**data_e.to_dict())
        except FunctionTimedOut as e1:
                d['__error'] = {
                    "error_type":"TIMEOUT",
                    "error_message": "extract function timeout",
                    "traceback":"TIMEOUT"
                }

                yield Row(**d)
        except Exception as e:
                # 记录更详细的错误信息
                error_info = {
                    "error_data":Row(**d),
                    "error_type": type(e).__name__,
                    "error_message": str(e),
                    "traceback": traceback.format_exc(),
                }
         
                # s3_doc_writer.write(error_info)
                d['__error'] = error_info

                yield Row(**d)
           




def extract_domain_info(url: str) -> dict:
    """从 URL 中提取完整的域名信息"""
    parsed = urlparse(url)
    netloc = parsed.netloc
    domain_parts = netloc.split(":")
    domain = domain_parts[0]  # 去除端口号
    root_domain = ".".join(domain.split(".")[-2:]) if len(domain.split(".")) >= 2 else domain

    return {
        "full_url": url,
        "netloc": netloc,
        "domain": domain,
        "root_domain": root_domain
    }
        
def handle_error(row: Dict, error: Exception) -> Dict:
    """统一错误处理"""
    row_dict = row.asDict()
    return {
     ** row,
    "__error": {
        "type": type(error).__name__,
        "message": str(error),
        "traceback": traceback.format_exc()
    }
    }

    
def extract_platform_from_s3_path(s3_path: str) -> str:
    """
    从 S3 路径中提取平台名称（存储桶后的第一个目录）
    
    示例输入: 
    - "s3://private-cooperate-data/zh-web-baijiahao/20241218_p1/"
    输出: "zh-web-baijiahao"
    
    - "s3://private-cooperate-data/DouBan/"
    输出: "DouBan"
    """
    # 分割路径并过滤空字符串
    parts = [p for p in s3_path.split("/") if p.strip() != ""]
    
    # 验证路径格式
    if len(parts) < 3:
        raise ValueError(f"无效的 S3 路径格式: {s3_path}")
    
    # 平台名称是存储桶后的第一个目录
    
    return parts[2]
# 定义判断是否为纯文本
def is_plain_text(row):
    html = row.get("html","")
    return not re.search(r'<[^>]+>', html)
    
# 定义判断content_list是否为空
def is_empty_content_list(row):
    content_list = row.get("content_list","")
    tag =False
    for i in content_list:
        if len(i) == 0:
            tag = True
    return tag
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, MapType, StringType
import re


def parse_text_paragraphs(row):
    # 按换行符分割段落，过滤空行
    html_content = row.get("html", "")
    paragraphs = []
    tmp_dict = {}
    for para in html_content.split('\n'):
        if para.strip():
            paragraphs.append({"c": para.strip(), "t": "text"})
        
    tmp_dict = {
            "type": "paragraph",
            "bbox": None,
            "raw_content": None,
            "content": paragraphs
  
        }      
    # 添加/更新content_list字段
    row["content_list"] = [[tmp_dict]]
    
    # 重建Row对象保留所有原始字段
    return row
   
  
import re,regex
from typing import List, Dict

def is_empty_content(text: str) -> bool:
    """检查文本是否为空或仅包含空白字符"""
    
    return not (text or "").strip()

def has_excessive_whitespace(text: str) -> bool:
    """检测过多空格或换行"""
    # 换行符占比超过25%
    total_chars = len(text)
    if total_chars == 0:
        return True
    newline_ratio = text.count('\n') / total_chars
    if newline_ratio > 0.25:
        return True
    
    # 连续空格超过500个或连续换行超过8个
    if ' ' * 500 in text or '\n' * 8 in text:
        return True
    
    return False

line_breaks_re = r"[\n\v\f\r\x85\u2028\u2029]"
visible_spaces_re = r"[\x20\xa0\u2000-\u200a\u202f\u205f\u3000]"
invisible_spaces_re = r"[\u200b-\u200d\u2060\ufeff]"
invisible_chars_re = r"[\xad\ufffc\u2061-\u2063]"
other_controls_re = r"[\x00-\x08\x0e-\x1f\x7f-\x84\x86-\x9f]"
direction_controls_re = r"[\u061c\u200e\u200f\u202a-\u202e\u2066-\u2069]"
head_view_invisible_spaces_re = r"^[\x20\xa0\u2000-\u200a\u202f\u205f\u3000 ]"
private_use_area_pattern = (
    r"[\uE000-\uF8FF]"  # BMP 私有使用区
    r"|[\U000F0000-\U000FFFFD]"  # 辅助平面 A 私有使用区
    r"|[\U00100000-\U0010FFFD]"  # 辅助平面 B 私有使用区
)
ar_invisible_spaces_re = r"[\u2060\ufeff]"
ar_direction_controls_re = r"[\u061c\u202c\u2066-\u2069]"
others = r"[\u2063\x00-\x1F\x7F-\x9F�\u200B-\u200D\uFEFF\u206a\u206e\u206f\u00AD\u200c\xa0\u3000\u2003\u2002\u200e\u00A0\u200e\u25A1\xa0]|&nbsp|\\? :|xa0|&ldquo;|&rdquo;|&rdquo|�|□|&amp|&lt|&gt"

def clean_special_whitespace(s):
    if s is None:
        return True
    s = str(s)
    s = re.sub(line_breaks_re, "\n", s)
    s = re.sub(visible_spaces_re, " ", s)
    s = re.sub(head_view_invisible_spaces_re, "", s)
    s = re.sub(invisible_spaces_re, "", s)
    s = re.sub(direction_controls_re, "", s)
    s = re.sub(invisible_chars_re, "", s)
    s = re.sub(other_controls_re, "", s)
    s = re.sub(private_use_area_pattern, "", s)
    s = re.sub(others,"",s,flags=re.UNICODE)
    s = regex.sub(r'\p{Z}+', ' ', s, flags=regex.UNICODE)
    # 匹配任意数量的指定空白字符（包括常规空格、\u2005、\u200c等）
    return s


def clean_html_tag(s):
    """处理 QUALITY_BAD_EFFECTIVENESS 类型数据质量问题"""
    # 模式选择：清理内容 或 严格过滤
    PROCESS_MODE = "clean"  # 可配置为 "filter" 进行严格过滤
    
    
    # 检测到 HTML 标签时才处理
    if bool(re.search(r'<[^>]+>', s)):
        if PROCESS_MODE == "clean":
            """清理 HTML 标签，保留文本内容"""
            s = re.sub(r'<\/?[a-z][^>]*>', '', s, flags=re.IGNORECASE)

    return  s



    
def is_url(text: str) -> bool:
    """判断文本是否为URL"""
    url_pattern = re.compile(
        r'^(?:http|ftp)s?://'  # 匹配http/https/ftp
        r'(?:\S+(?::\S*)?@)?'  # 用户名密码
        r'(?:\d{1,3}\.){3}\d{1,3}'  # IPv4地址
        r'|'                     # 或域名
        r'(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}'  # 域名
        r'(?::\d+)?'            # 端口
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return bool(re.match(url_pattern, text))


def is_only_url(text: str) -> bool:
    """判断字符串是否仅包含一个完整URL且无其他内容"""
    # 优化后的正则表达式，支持常见URL格式
    url_pattern = re.compile(
        r'^'                            # 字符串开始
        r'(https?|ftp)://'              # 协议头
        r'(?:\S+(?::\S*)?@)?'           # 用户名密码（可选）
        r'([a-z0-9\-]+\.)+[a-z]{2,}'    # 域名部分
        r'(?::\d+)?'                    # 端口号（可选）
        r'(?:/[^\s?#]*)?'               # 路径（可选）
        r'(?:\?[^\s#]*)?'               # 查询参数（可选）
        r'(?:#[^\s]*)?'                 # 锚点（可选）
        r'$',                           # 字符串结束
        re.IGNORECASE
    )
    return url_pattern.fullmatch(text.strip()) is not None

def is_too_short(text: str) -> bool:
    """判断文本长度是否过短"""
    return len(text) <= 21

def has_abnormal_chars(text: str) -> bool:
    """检测异常字符（如不可见字符、�）"""
    abnormal_pattern = re.compile(r'[\x00-\x1F\x7F-\x9F�\u200B-\u200d\uFEFF\u00AD\u200c\xa0\u3000\u2003\u2002\u200e]')
    return bool(abnormal_pattern.search(text))

    


def clean_repeat_chars(text, max_repeat=3):
    """
    清理无意义重复字符
    :param text: 原始文本
    :param max_repeat: 允许的最大连续重复次数(中文建议3-5)
    :return: 清洗后文本
    """
    # 匹配超过阈值的连续重复字符（支持全角/半角）
    pattern = rf"([\w\W])\1{{{max_repeat},}}"
    
    def replace_func(match):
        char = match.group(1)
        # 保留不超过允许的重复次数
        return char * max_repeat
    
    return re.sub(pattern, replace_func, text)



import re
from typing import List, Dict

def is_single_url(text: str) -> bool:
    """严格判断是否为单个URL（修复括号平衡）"""
    cleaned_text = text.strip()
    url_pattern = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.IGNORECASE)
    return url_pattern.fullmatch(cleaned_text) is not None

def is_single_url(text: str) -> bool:
    URL_PATTERN = re.compile(
        r'^'
        r'(?:(?:https?):\/\/)?'  # 协议头（非捕获组优化）
        r'(?:www\.)?'            # 可选www前缀（网页2非捕获组建议）
        r'(?:'                   # 主域名部分（原子组优化）
            r'(?:[a-zA-Z0-9-]{1,63}\.)+[a-zA-Z]{2,63}'  # 域名结构（网页1长度限制）
            r'|'                                        # 或IP地址
            r'(?:\d{1,3}\.){3}\d{1,3}'                 # IPv4（精确匹配）
            r'|'                                        # 或
            r'$$([a-f0-9:]+)$$'                        # IPv6（网页4特殊符号处理）
        r')'
        r'(?::\d{1,5})?'         # 端口号（限制1-5位数字）
        r'(?:\/'                 # 路径参数（网页3回溯预防）
            r'[\w$-_.+!*\'(),%;&=@~#]*'  # 安全字符集（排除回溯风险）
            r'(?:%[0-9a-fA-F]{2})*'      # 编码字符（替代宽泛匹配）
        r')*'                   # 允许多级路径
        r'$', 
        re.IGNORECASE | re.VERBOSE
    )

    """严格验证单个URL（支持IPv6/Punycode/特殊符号）"""
    cleaned = text.strip() # 清理边界符号（网页4的清理建议）
    url_pattern_2 = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.IGNORECASE)
    tag = URL_PATTERN.fullmatch(cleaned) is not None
    tag2 = url_pattern_2.fullmatch(cleaned) is not None
    logger.info(cleaned)
    return tag2


def is_url_only_content(content: str) -> bool:
    """判断内容是否全部由URL构成（改进分割逻辑）"""
    if not content.strip():
        return False
    
    # 分割策略：仅按空白符（空格、换行等）分割，保留URL内部字符
    segments = re.split(r"""\n+|[\s\'">]+|'""", content.strip())
    return all(
        is_single_url(segment)
        for segment in segments
        if segment.strip()  # 过滤空段落
    )

import re
from typing import Dict, List

import traceback
def is_valid_content_element(processed: Dict) -> Dict:
    """综合校验单个content元素的有效性"""
    # 基础校验：字段完整性
    # 优先取 'c'，若不存在则取 'title_content'，最后默认空字符串
   
    content_key = 'c' if 'c' in processed else 'title_content' if 'title_content' in processed else 'c'

    raw_content = processed.get(content_key, '')
    elem_type = processed.get('t', 'text') 
    

    
    # 规则1: 空内容或纯空格
    if is_empty_content(raw_content):
        processed[content_key] = ''
        processed['t'] = 'text'  # 强制类型为text
        return processed
    
    
    # 规则2: 过多空白（根据不同类型调整阈值）
    if has_excessive_whitespace(raw_content):
        processed[content_key] = ''
        processed['t'] = 'text'  # 强制类型为text
        return processed

 
    # --- 清理逻辑 ---
    cleaned = clean_html_tag(clean_repeat_chars(clean_special_whitespace(raw_content)))
  
    processed[content_key] = cleaned  # 更新原始内容键
    processed['t'] = elem_type  # 保留原始类型或默认值
       
    return processed


from typing import List, Dict

def filter_content(content: List[Dict]) -> List[Dict]:
    """过滤content列表，保留有效内容元素"""
    filtered = []
    for elem in content:
        # 步骤1: 校验并处理单个元素
        processed = is_valid_content_element(elem)
        
        # 步骤2: 动态判断内容键（优先'c'，其次是'title_content'）
        content_key = 'c' if 'c' in processed else 'title_content' if 'title_content' in processed else None
        if not content_key:
            continue  # 无内容键，直接跳过
        
        # 步骤3: 检查内容是否有效（非空且非纯空格）
        content_value = processed.get(content_key, '')
        if content_value.strip() != '':
            filtered.append(processed)
            
    return filtered

def update_value(data, target_key, condition = lambda x :True, new_value =clean_special_whitespace):
    """
    递归地遍历任意维的字典列表堆叠结构数据，并根据条件更新特定键的值。

    :param data: 数据结构（字典或列表）
    :param target_key: 要更新的目标键
    :param condition: 更新条件函数
    :param new_value: 新的值（可以是具体值或函数）
    """
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key and condition(value):
                if callable(new_value):
                    data[key] = new_value(value)
                else:
                    data[key] = new_value
            else:
                update_value(value, target_key, condition, new_value)
    elif isinstance(data, list):
        for item in data:
            update_value(item, target_key, condition, new_value)
   
        
def filter_content_list(row):
    """优化点：分离标题/段落处理，防御性类型检查，保留原始数据结构"""
    try:
        # 防御性获取内容列表
        for page in row.get("content_list", []):
            valid_elements = []
            for elem in page:
                # 非字典或非目标类型直接保留
                if not isinstance(elem, dict) or elem.get('type') not in ('title', 'paragraph','list'):
                    valid_elements.append(elem)
                    continue

                # 类型分流处理
                elem_type = elem['type']

                content_data = elem.get('content')

                # --- 标题类型处理 ---
                if elem_type == 'title':
                    # 标题内容必须是字典且包含 title_content
                    if isinstance(content_data, dict) and 'title_content' in content_data:
                        filtered = filter_content([content_data])  # 包装成列表
                        # 校验返回结果有效性
                        if filtered and isinstance(filtered, list) and len(filtered) > 0:
                            new_content = filtered[0]
                            # 确保标题内容非空
                            if new_content.get('title_content', '').strip():
                                elem['content'] = new_content
                                valid_elements.append(elem)
                    # 无效标题内容直接过滤
                    continue

                # --- 段落类型处理 ---
                if elem_type == 'paragraph':
                    # 段落内容必须是列表
                    if isinstance(content_data, list):
                        filtered = filter_content(content_data)
                        # 保留非空内容
                        if filtered and len(filtered) > 0:
                            elem['content'] = filtered
                            valid_elements.append(elem)
                    # 无效段落内容直接过滤
                    continue
                if elem_type == 'list':
                    update_value(content_data,'c')
                  
                    elem['content'] = content_data
                   
                    valid_elements.append(elem)
                    


            # 原地更新页面内容
            page[:] = valid_elements
        return row
    except Exception as e:
        row['__error'] = {
            'code': -1,
            'msg': f"Content filtering failed: {str(e)}",
            'trace': traceback.format_exc()
        }
        return row
from llm_web_kit.input.datajson import DataJson

def make_content_md(row):
    try:
        row['content_qa'] = DataJson(row).get_content_list().to_nlp_md()

        return row
    except Exception as e2:
        err = {"code":-1, f"{row}msg": traceback.format_exc()}
        row['content_qa'] = ''
        row['__error'] = err
        return row

def del_content_qa(row):
    del row['content_qa']
    return row

from llm_web_kit.input.datajson import DataJson, StructureMapper
def clist_filter_factory(self, field_you_want = ('paragraph')):
    """把content_list转化为md格式，只接受 Node 类型为 ['a', 'b'].
    Returns:
        str: md格式的文本内容
    """
    md_blocks = []  # 每个是个DocElementType规定的元素块之一转换成的文本
    content_lst = self._get_data()
    for page in content_lst:
        for content_lst_node in page:
            if content_lst_node['type'] in field_you_want:
                txt_content = self._StructureMapper__content_lst_node_2_md(content_lst_node)
                if txt_content and len(txt_content) > 0:
                    md_blocks.append(txt_content)
    md = self._StructureMapper__md_para_splitter.join(md_blocks)
    md = md.strip() + self._StructureMapper__text_end  # 加上结尾换行符
    return md

    
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
#安全解析函数（包含异常捕获）
def safe_json_loads(s):
    try:
        json.loads(s)
        return True
    except:
        return False
# 将Python函数注册为Spark UDF
safe_json_udf = udf(safe_json_loads, BooleanType())
             


In [5]:
version="002"
sub_dir=''
input_paths =[f's3://crawl-data/blog_sina_com_cn/gz_file/']
platform = extract_platform_from_s3_path(input_paths[0])
platform_configs = {

    "zh-web-baijiahao": {

        "field_mappings": {
            "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video"
                            },
        "dataset_name": "baijiahao",
        "url":"url",
        "html":"content",
        "layout_field": "channel",
        "data_source_category":"JSON"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/{sub_dir}/v{version}/"

    },
    "zh-web-netease": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video"
                            },
        "dataset_name": "net-ease",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "zh-web-tencent": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "腾讯网":"article"
                            },
        "dataset_name": "tencent",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
     "zh-web-sohu": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article"
                            },
        "dataset_name": "souhu",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "zh-web-sina": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum",
                                "新浪网":"article"
                            },
        "dataset_name": "sina",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },

    "blog_sina_com_cn": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum",
                                
                            },
        "dataset_name": "sina_blog",
        "url":"url",
        "html":"html",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "DouBan": {
       
        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum",
                                "豆瓣网":""
                            },
        "dataset_name": "sina",
        "url":"url",
        "html":"html",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
}
platform_config = platform_configs.get(platform)
platform_config


{'field_mappings': {'track_id': 'track_id',
  'page_layout_type_map': {'': 'article',
   '文章': 'article',
   '网易': 'article',
   '视频': 'video',
   '搜狐网': 'article',
   '黑猫投诉': 'forum'},
  'dataset_name': 'sina_blog',
  'url': 'url',
  'html': 'html',
  'layout_field': 'f_name',
  'data_source_category': 'HTML'},
 'extractor_config': '/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc',
 'output_template': 's3://llm-users-phdd2/jiangwenhao/article/blog_sina_com_cn/v002/'}

In [6]:
config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    # 根据个人路径进行替换1
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
    "spark.yarn.queue": "root.clean_exp"
}
spark = new_spark_session("llm_kit_cc", config)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:

print(f"路径: {input_paths[0]} → 平台: {platform}")


"""接收 spark 作为参数而不是持有它"""
platform_config = platform_configs.get(platform)
if not platform_config:
    raise ValueError(f"Unsupported platform: {platform}")

# Driver 端操作
input_df = read_any_path(spark, ",".join(input_paths), config)
print(f"读取数据结束")
# 准备 Worker 端配置
worker_config = {
    "field_mappings": platform_config.get("field_mappings"),
    "extractor_config": platform_config.get("extractor_config")
}
# 分别广播不同配置
broadcast_field_mappings = worker_config["field_mappings"]
broadcast_extractor_config_path = worker_config["extractor_config"] # 广播路径






#processed_rdd_error_filter = processed_rdd.filter(lambda x:"__error"  in x)

#write_any_path(processed_rdd_error_filter.map(lambda x: Row(value=json_dumps(x.asDict()))).toDF(),"s3://web-parse-hw60p/xuchao/zx-html-error/tencent_blog/v002")


#result_df = processed_rdd_filter.map(lambda x: Row(value=json_dumps(x.asDict()))).toDF()

# write_any_path(result_df, platform_config["output_template"])

路径: s3://crawl-data/blog_sina_com_cn/gz_file/ → 平台: blog_sina_com_cn
读取数据结束


In [8]:


# 在filter中使用UDF生成的Column表达式
processed_rdd_filter = input_df.filter(safe_json_udf(col("value"))).rdd.map(lambda x: Row(**{**json.loads(x.value), "filename": x.filename})).map(
    lambda row: transform_row(row, broadcast_field_mappings)
).repartition(6000).filter(lambda x:"__error" not in x).filter(lambda x :x.html !='').mapPartitions(
     lambda x:extract_data(
        x,broadcast_extractor_config_path = broadcast_extractor_config_path
    )
).filter(lambda x:"__error" not in x).cache()
#\input_path = ["s3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v002/"]
#input_df = read_any_path(spark, ",".join(input_path), config)
processed_rdd_filter = processed_rdd_filter.map(lambda x :x.asDict())




In [9]:
spark

In [10]:
filter_json_rdd = processed_rdd_filter.filter(is_plain_text)


filter_html_empty_rdd = processed_rdd_filter.filter(lambda x : is_empty_content_list(x) and not is_plain_text(x))

## 是HTML但是content_list不为空的数据集==》正常输出数据集
filter_html_rdd = processed_rdd_filter.filter(lambda x : not is_empty_content_list(x) and not is_plain_text(x))

filter_json_add_content_list_rdd = filter_json_rdd.map(lambda x : parse_text_paragraphs(x))

filter_html_add_content_list_rdd = filter_html_empty_rdd.map(parse_text_paragraphs)



union_rdd = filter_html_rdd.union(filter_html_add_content_list_rdd).union(filter_json_add_content_list_rdd)
final_rdd = union_rdd.map(filter_content_list).filter( lambda x : not is_empty_content_list(x)).map(make_content_md)


In [11]:
final_rdd_1 = final_rdd.filter(lambda x: not  is_too_short(x['content_qa']))
final_rdd_2 = final_rdd_1.filter(lambda x : not has_excessive_whitespace(x['content_qa']))



In [12]:
#from loguru import logger
#        URL_PATTERN = re.compile(
#            r'^'
#            r'(?:(?:https?):\/\/)?'  # 协议头（非捕获组优化）
#            r'(?:www\.)?'            # 可选www前缀（网页2非捕获组建议）
#            r'(?:'                   # 主域名部分（原子组优化）
#                r'(?:[a-zA-Z0-9-]{1,63}\.)+[a-zA-Z]{2,63}'  # 域名结构（网页1长度限制）
#                r'|'                                        # 或IP地址
#                r'(?:\d{1,3}\.){3}\d{1,3}'                 # IPv4（精确匹配）
#                r'|'                                        # 或
#                r'$$([a-f0-9:]+)$$'                        # IPv6（网页4特殊符号处理）
#            r')'
#            r'(?::\d{1,5})?'         # 端口号（限制1-5位数字）
#            r'(?:\/'                 # 路径参数（网页3回溯预防）
#                r'[\w$-_.+!*\'(),%;&=@~#]*'  # 安全字符集（排除回溯风险）
#                r'(?:%[0-9a-fA-F]{2})*'      # 编码字符（替代宽泛匹配）
#            r')*'                   # 允许多级路径
#            r'$', 
#            re.IGNORECASE | re.VERBOSE
#        )
#    
#        """严格验证单个URL（支持IPv6/Punycode/特殊符号）"""
#        cleaned = text.strip() # 清理边界符号（网页4的清理建议）
#        url_pattern_2 = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.IGNORECASE)
#        tag = URL_PATTERN.fullmatch(cleaned) is not None
def is_single_url(text: str) -> bool:
    cleaned = text.strip()
    
    # 前置条件：必须以http/https或www开头（不区分大小写）
    lower_clean = cleaned.lower()
    if not (lower_clean.startswith(('http://', 'https://', 'www.'))):
        return False
    else:
        return True
        
#  tag2 = url_pattern_2.fullmatch(cleaned) is not None
    



def is_url_only_content_2(content: str) -> bool:
    """判断内容是否全部由URL构成（改进分割逻辑）"""
    if not content.strip():
        return False
    
    # 分割策略：仅按空白符（空格、换行等）分割，保留URL内部字符
    segments = re.split(r"""\n+|[\s\'">]+|'""", content.strip())
    return all(
        is_single_url(segment)
        for segment in segments
        if segment.strip()  # 过滤空段落
    )


In [13]:
final_rdd_3 = final_rdd_2.filter(lambda x : not is_url_only_content_2(x['content_qa']))


In [14]:
from pyspark import SparkConf

from dingo.model.model import Model
from dingo.io import InputArgs
from dingo.exec import Executor
from dingo.io import MetaData

In [15]:
def testing(row):
    d= {'track_id': row['track_id'],'content_qa': DataJson(row).get_content_list().to_nlp_md()}
    return d

In [16]:
input_rdd_format = final_rdd_3.map(lambda x : testing(x))

input_rdd_format = input_rdd_format.map(lambda x: MetaData(
    data_id= str(x['track_id']),
    prompt='',
    content=x['content_qa'],
    # image=json.loads(x.value)['image'],
    # raw_data = json.loads(x.value)
))

input_data = {
    "eval_group": "qa_standard_v1",
    'input_path': 'redpajama',
    'save_data': True,
    # 'save_raw': True
}
# input_data = {
#     "eval_group": "test",
#     "input_path": "redpajama",  # local filesystem dataset
#     "save_data": True,
#     "custom_config":
#         {
#             "rule_list": ["CommonSpecialCharacter", "CommonColonEnd"],
#             "prompt_list": ["LLMRepeat"],
#             "llm_config":
#                 {
#                     "detect_text_quality":
#                         {
#                             "key": "sk-proj-CwgXjGGBKq9vBI0Lue3jT3BlbkFJ5GnKDvQtrCmlwdHoea04",
#                             "api_url": "http://10.140.54.48:26666/v1",
#                         }
#                 }
#         }
# }
# Model.apply_config(input_data['custom_config_path'])
input_args = InputArgs(**input_data)
executor = Executor.exec_map["spark"](input_args, spark_session=spark, spark_rdd=input_rdd_format)
result = executor.execute()
result = result[0].to_dict()
print(result)

============= Init pyspark =============
============== Init Done ===============


[Stage 15:============================================>   (16561 + 784) / 18000]

{'task_id': 'b0a1bb5e-047a-11f0-8755-e8ebd34ed3cc', 'task_name': 'dingo', 'eval_group': 'qa_standard_v1', 'input_path': '', 'output_path': '', 'create_time': '20250319_114923', 'finish_time': '20250319_122924', 'score': 100.0, 'num_good': 27359702, 'num_bad': 1088, 'total': 27360790, 'type_ratio': {'QUALITY_BAD_EFFECTIVENESS': 4e-05}, 'name_ratio': {'QUALITY_BAD_EFFECTIVENESS-RuleAbnormalChar': 3.4e-05, 'QUALITY_BAD_EFFECTIVENESS-RuleAbnormalHtml': 5e-06}}


In [15]:
executor.bad_info_list.filter(lambda x :x['name_list'] ==['QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl']).take(500)

[]

In [17]:
final_df = final_rdd_3.map(lambda x: Row(value=json_dumps(x))).toDF()

In [18]:

# 验收数据
#腾讯
write_any_path(final_df, "s3://zhuanxiang-hw60p/article/blog_sina_com_cn/v002/")
#souhu
#write_any_path(final_df, "s3://zhuanxiang-hw60p/article/souhu/final/v002/")
#sina
#write_any_path(final_df, "s3://zhuanxiang-hw60p/article/sina/v008/")
#sina——blog
#write_any_path(final_df, "s3://zhuanxiang-hw60p/article/blog_sina_com_cn/final/v001/")


{'rows': 27360790,
 'bytes': {'sum': 1327047431876,
  'min': 778,
  'max': 3961965,
  'cnt': 27360790,
  'avg': 48501.795},
 'files': 12000,
 'sub_paths': {}}

In [ ]:
1

25/03/18 13:33:55 WARN TaskSetManager: Lost task 7706.0 in stage 90.0 (TID 40632) (host-10-140-92-191 executor 290): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 14145.0 in stage 90.0 (TID 47070) (host-10-140-93-38 executor 187): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 10929.0 in stage 90.0 (TID 43855) (host-10-140-92-24 executor 176): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 13322.0 in stage 90.0 (TID 46247) (host-10-140-92-60 executor 65): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 12328.0 in stage 90.0 (TID 45253) (host-10-140-92-140 executor 146): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 12352.0 in stage 90.0 (TID 45277) (host-10-140-92-151 executor 204): TaskKilled (Stage cancelled)
25/03/18 13:33:55 WARN TaskSetManager: Lost task 13388.0 in stage 90.0 (TID 46313) (host-10-140-92-140 executor 148): TaskK